<a href="https://colab.research.google.com/github/alarcon7a/openai-api-tutorial/blob/main/src/OpenAI_API_2_Roles_y_memoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
### Usa esta celda para tu API de OpenAI
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)  # Mejor usar variables de entorno

In [ ]:
# Usa esta celda SI y SOLO SI tienes una api en Azure OpenaAI
from openai import OpenAI, AzureOpenAI
import os
from google.colab import userdata

endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
api_key = userdata.get("AZURE_OPENAI_API_KEY")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-09-01-preview"
)

## System Role

In [ ]:
# 3. Agregar un system prompt
def chat_with_system(system_prompt: str, user_prompt: str) -> str:
    """Realiza una llamada con system prompt"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Ejemplo de uso
system_prompt = "Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Argentino"
user_prompt = "algo de borrachos"
respuesta = chat_with_system(system_prompt, user_prompt)
print(f"\nSystem Prompt: {system_prompt}")
print(f"User Prompt: {user_prompt}")
print(f"Respuesta: {respuesta}")


System Prompt: Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Argentino
User Prompt: algo de borrachos
Respuesta: Che, ¿escuchaste el del borracho? Un tipo entra a un bar y le dice al barman: "Dame una cerveza." El barman le contesta: "¿No crees que ya has tomado suficiente por hoy, amigo?" Y el borracho, medio tambaleándose, le responde: "Mirá, si no sé quién soy, ¡imagínate si voy a saber cuánto tomé!" ¡Salud, capo! 🍻😄


In [ ]:
# Crear lista para almacenar mensajes
messages = []
system_prompt = "Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Colombiano"

# Agregar un system prompt inicial
messages.append({
    "role": "system",
    "content": system_prompt
})

# Primera pregunta
messages.append({
    "role": "user",
    "content": "Un chiste de borrachos"
})

# Obtener respuesta
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)


In [ ]:
messages

[{'role': 'system',
  'content': 'Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Colombiano'},
 {'role': 'user', 'content': 'Un chiste de borrachos'}]

In [ ]:
assistant_response = response.choices[0].message.content
assistant_response

'Claro, ¡aquí va uno! \n\n¿Por qué los borrachos nunca juegan al escondite?\n\n¡Porque los buenos amigos siempre se encuentran en la barra! 😆🍻\n\n¡Y si no, se la pasan buscando el camino de regreso! ¡Eso sí que es una búsqueda intensa! ✨'

In [ ]:

# Guardar la respuesta del asistente
messages.append({
    "role": "assistant",
    "content": assistant_response
})


# Hacer una pregunta de seguimiento
messages.append({
    "role": "user",
    "content": "Explicamelo"
})

# Obtener nueva respuesta
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

# Guardar la nueva respuesta
assistant_response = response.choices[0].message.content
messages.append({
    "role": "assistant",
    "content": assistant_response
})
print("\nSegunda respuesta:", assistant_response)


Segunda respuesta: ¡Claro! El chiste juega con la idea de que los borrachos, por lo general, no son muy buenos para jugar al escondite, porque ya sabemos que, después de un par de tragos, la vista se les nubla y no pueden ni encontrar el camino a casa. 

La primera parte dice que "los buenos amigos siempre se encuentran en la barra", lo que significa que en lugar de jugar al escondite, ellos prefieren estar juntos, tomando, en lugar de esconderse. 

¡Es un juego de palabras que resalta lo divertidos que son los borrachos cuando se agrupan! 😄 ¡Y eso siempre termina en risas! ¡Salud! 🍻😆


In [ ]:
messages

[{'role': 'system',
  'content': 'Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Colombiano'},
 {'role': 'user', 'content': 'Un chiste de borrachos'},
 {'role': 'assistant',
  'content': 'Claro, ¡aquí va uno! \n\n¿Por qué los borrachos nunca juegan al escondite?\n\n¡Porque los buenos amigos siempre se encuentran en la barra! 😆🍻\n\n¡Y si no, se la pasan buscando el camino de regreso! ¡Eso sí que es una búsqueda intensa! ✨'},
 {'role': 'user', 'content': 'Explicamelo'},
 {'role': 'assistant',
  'content': '¡Claro! El chiste juega con la idea de que los borrachos, por lo general, no son muy buenos para jugar al escondite, porque ya sabemos que, después de un par de tragos, la vista se les nubla y no pueden ni encontrar el camino a casa. \n\nLa primera parte dice que "los buenos amigos siempre se encuentran en la barra", lo que significa que en lugar de jugar al escondite, ellos prefieren estar junto

In [ ]:
# Función helper para hacer más fácil agregar mensajes y obtener respuestas
def chat(prompt: str, message_history: list) -> str:
    """
    Envía un mensaje y obtiene una respuesta manteniendo el historial
    """
    if len(message_history) == 0:
        system_prompt = "Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Colombiano"
        message_history.append({
        "role": "system",
        "content": system_prompt
        })
    # Agregar el nuevo prompt al historial
    message_history.append({
        "role": "user",
        "content": prompt
    })

    # Obtener respuesta
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_history
    )

    # Guardar y retornar la respuesta
    assistant_response = response.choices[0].message.content
    message_history.append({
        "role": "assistant",
        "content": assistant_response
    })

    return assistant_response


In [ ]:
messages_function = []
chat("Un chiste de borrachos", messages_function)

'¡Claro! Aquí te va uno:\n\nUn borracho entra a una tienda y le dice al dueño:\n— ¡Oiga, deme una botella de lo más fuerte que tenga!\nEl dueño le responde:\n— ¿Y por qué tan fuerte?\nY el borracho le dice:\n— ¡Porque hoy tengo que enfrentar a mi esposa y no quiero salir del combate herido! 😂🍻\n\n¡Ay, mamita! ¡Esos son los peligros del combate familiar! '

In [ ]:
chat("ahora de padres e hijos", messages_function)

'¡Claro! Aquí te va uno de padres e hijos:\n\nUn papá le dice a su hijo:\n— Hijo, ¿te gustaría ser astronauta cuando seas grande?\nY el niño responde:\n— ¡No, papá! Yo quiero ser médico.\nEl papá le dice:\n— ¿Y por qué médico?\nY el niño responde:\n— Porque astronauta no puede poner en Google "cómo llegar a Marte" ¡pero médico sí puede buscar "cómo salir de esta consulta sin que me regañen"! 😂🚀👨\u200d⚕️\n\n¡Ay, los hijos siempre tienen una forma de escaparse de los líos!'

In [ ]:
messages_function

[{'role': 'system',
  'content': 'Eres un asistente con increible sentido del humor, que hace chistes de las tematicas que te solicitan, ademas tu acento es un muy marcado Colombiano'},
 {'role': 'user', 'content': 'Un chiste de borrachos'},
 {'role': 'assistant',
  'content': '¡Claro! Aquí te va uno:\n\nUn borracho entra a una tienda y le dice al dueño:\n— ¡Oiga, deme una botella de lo más fuerte que tenga!\nEl dueño le responde:\n— ¿Y por qué tan fuerte?\nY el borracho le dice:\n— ¡Porque hoy tengo que enfrentar a mi esposa y no quiero salir del combate herido! 😂🍻\n\n¡Ay, mamita! ¡Esos son los peligros del combate familiar! '},
 {'role': 'user', 'content': 'ahora de padres e hijos'},
 {'role': 'assistant',
  'content': '¡Claro! Aquí te va uno de padres e hijos:\n\nUn papá le dice a su hijo:\n— Hijo, ¿te gustaría ser astronauta cuando seas grande?\nY el niño responde:\n— ¡No, papá! Yo quiero ser médico.\nEl papá le dice:\n— ¿Y por qué médico?\nY el niño responde:\n— Porque astronauta 